In [1]:
!pip install selenium

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import tqdm

In [3]:
browser = Chrome("/Users/kristinaastakhova/documents/JupyterLabProjects/chromedriver")

/var/folders/_3/rv5xs0111_x9xxfh084170rw0000gn/T/ipykernel_18626/2791085424.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome("/Users/kristinaastakhova/documents/JupyterLabProjects/chromedriver")


In [4]:
url = 'https://www.hse.ru/org/persons/?ltr=%D0%90;udept=22726'

In [5]:
browser.get(url)
# browser.page_source

In [6]:
#достаем инфо о профессорах и проверяем длинну
profs = browser.find_elements(by=By.CLASS_NAME, value='post__content_person')
# contact_info = browser.find_elements(by=By.CLASS_NAME, value='l-extra')
# names = browser.find_elements(by=By.CLASS_NAME, value='b')
# job_info = browser.find_elements(by=By.CLASS_NAME, value='with-indent7')
# print(len(profs), len(contact_info), len(names), len(job_info))
print(len(profs))

468


In [8]:
data = []
number = 0

for prof in tqdm.tqdm(profs):
    
    # получаем массив с инфо о контактах
    contact_info = prof.find_elements(by=By.CLASS_NAME, value='l-extra')
    
    try:
    # нам нужен первый элемент массива
        contacts = contact_info[0].text.split('\n')
    except:
        contacts = '-'
       
    try:
    # получаем массив с инфо об имени, нам нужен первый элемент массива
        name = prof.find_elements(by=By.CLASS_NAME, value='b')[0].text
    except:
        name = '-'
        print(number,'Проверить вручную имя')

    # создаем пустые массивы, куда положим инфо о должностях и местах
    statuses = []
    places = []
    
    job_info = prof.find_elements(by=By.CLASS_NAME, value='with-indent7')
    
    try:
        for jobs in job_info:
            # разделяем работы
            jobs = jobs.text.split('\n')
            for job in jobs:
                # разделяем должность и место
                job = job.split(':')
                status = job[0]
                place = job[1]
                statuses.append(status)
                places.append(place)
    except:
        status = '-'
        place = '-'
        print(number, 'Проверить вручную должность и место')
        
        # кладем должности и места в массивы
        places.append(place)
        statuses.append(status)
    
    data.append([number, name, statuses, places, contacts])
    
    # обновляем счетчик
    number += 1

 18%|███████▋                                  | 85/468 [00:04<00:15, 24.57it/s]

80 Проверить вручную должность и место


 30%|████████████▏                            | 139/468 [00:06<00:13, 24.80it/s]

135 Проверить вручную должность и место


 61%|█████████████████████████                | 286/468 [00:12<00:07, 23.50it/s]

...

 84%|██████████████████████████████████▎      | 391/468 [00:17<00:03, 23.70it/s]

387 Проверить вручную должность и место


 99%|████████████████████████████████████████▌| 463/468 [00:20<00:00, 23.55it/s]

459 Проверить вручную должность и место


100%|█████████████████████████████████████████| 468/468 [00:20<00:00, 22.70it/s]


In [9]:
# кладем в датафрейм

header = ['number', 'name', 'statuses', 'places', 'contacts']
df = pd.DataFrame(data, columns=header)
df.head()

,number,name,statuses,places,contacts
0,0,Абалмасова Екатерина Сергеевна,"[стажер-исследователь, Приглашенный преподават...",[ Институт образования / Проектно-учебная лабо...,[22126]
1,1,Абанкина Ирина Всеволодовна,"[Профессор, главный научный сотрудник]",[ Институт образования / Департамент образоват...,[+7(495) 772-9590 *22073]
2,2,Абанкина Татьяна Всеволодовна,[Директор центра],[ Факультет городского и регионального развити...,[+7(495) 772-9590 *12036]
3,3,Абанокова Ксения Руслановна,[Младший научный сотрудник],[ Институт социальной политики / Центр комплек...,"[+7(495) 624-6240, +7(495) 772-9590]"
4,4,Абашин Сергей Николаевич,[Приглашенный преподаватель],[ Факультет гуманитарных наук / Институт класс...,[23222]


In [10]:
# достаем массивы с ссылками
link_info = browser.find_elements(by=By.CLASS_NAME, value='large')

In [11]:
# достаем ссылки
links = []

for el in link_info:
    el = el.get_attribute('href')
    links.append(el)

In [12]:
# проверяем количество ссылок
len(links)

468

In [13]:
hse_exp = []
number = 0

for link in tqdm.tqdm(links):
    browser.get(link)

    # достаем инфо о работе во ВШЭ
    try:
        hse = browser.find_elements(by=By.CLASS_NAME, value='person-employment-addition')[0].text.split('\n')[0]  
    except:
        hse='-'
        
    # пополяем массив
    hse_exp.append([number, link, hse])

    number += 1

100%|█████████████████████████████████████████| 468/468 [20:10<00:00,  2.59s/it]


In [14]:
header = ['number', 'link', 'in_hse']
df_1 = pd.DataFrame(hse_exp, columns=header)
df_1.head()

,number,link,in_hse
0,0,https://www.hse.ru/staff/esabalmasova,Начала работать в НИУ ВШЭ в 2017 году.
1,1,https://www.hse.ru/org/persons/25477,Начала работать в НИУ ВШЭ в 2001 году.
2,2,https://www.hse.ru/org/persons/203662,Начала работать в НИУ ВШЭ в 2001 году.
3,3,https://www.hse.ru/org/persons/14243866,Начала работать в НИУ ВШЭ в 2011 году.
4,4,https://www.hse.ru/org/persons/346247135,Начал работать в НИУ ВШЭ в 2021 году.


In [15]:
# соединяем два датафрейма по колонке с номером
df = df.merge(df_1, on=['number'])

In [16]:
# удаляем лишнюю колонку с номером
df = df.drop(columns = ['number'], axis = 1)
df.head()

,name,statuses,places,contacts,link,in_hse
0,Абалмасова Екатерина Сергеевна,"[стажер-исследователь, Приглашенный преподават...",[ Институт образования / Проектно-учебная лабо...,[22126],https://www.hse.ru/staff/esabalmasova,Начала работать в НИУ ВШЭ в 2017 году.
1,Абанкина Ирина Всеволодовна,"[Профессор, главный научный сотрудник]",[ Институт образования / Департамент образоват...,[+7(495) 772-9590 *22073],https://www.hse.ru/org/persons/25477,Начала работать в НИУ ВШЭ в 2001 году.
2,Абанкина Татьяна Всеволодовна,[Директор центра],[ Факультет городского и регионального развити...,[+7(495) 772-9590 *12036],https://www.hse.ru/org/persons/203662,Начала работать в НИУ ВШЭ в 2001 году.
3,Абанокова Ксения Руслановна,[Младший научный сотрудник],[ Институт социальной политики / Центр комплек...,"[+7(495) 624-6240, +7(495) 772-9590]",https://www.hse.ru/org/persons/14243866,Начала работать в НИУ ВШЭ в 2011 году.
4,Абашин Сергей Николаевич,[Приглашенный преподаватель],[ Факультет гуманитарных наук / Институт класс...,[23222],https://www.hse.ru/org/persons/346247135,Начал работать в НИУ ВШЭ в 2021 году.


In [17]:
# записываем в csv
df.to_csv("/Users/kristinaastakhova/documents/JupyterLabProjects/homework/DJ/profesors_2.csv")